In [1]:
import numpy as np

In [8]:
'''Activation Functions (활성함수들)'''

def sigmoid (x: np.array):
    return 1 / (1 + np.exp(-x))

def relu (x: np.array):
    return np.maximum(0, x)

def softmax(x: np.array):

    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T

    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

In [11]:
'''Loss Function (Cross-Entropy Error)'''

def cross_entropy_error(y: np.array, t: np.array):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]

    return -1 * (np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size)

In [ ]:
np.nditer

In [15]:
'''Numerical Gradient'''

def numerical_gradient(f, x:np.array):
    h = 1e-4
    grads = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])

    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        # f(x+h)
        x[idx] = tmp_val + h
        fx1 = f(x)
        # f(x-h)
        x[idx] = tmp_val - h
        fx2 = f(x)

        grads[idx] = (fx1 - fx2) / (2*h)

        x[idx] = tmp_val
        it.iternext()

    return grads

활성화 함수의 종류에 맞게, 순전파, 역전파 수행 계층 클래스 만들기

1. Relu

In [16]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x:np.array):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        '''mask 이용해서 음수는 0으로 날리는 거?'''
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

2. Sigmoid 계층

In [17]:
class sigmoid:
    def __init__(self):
        self.y = None
    
    def forward(self, x:np.array):
        y = sigmoid(x)
        self.y = y
        return y

    def backward(self, dout):
        dx = dout * self.y * ( 1 - self.y)
        return dx

행렬 곱 연산 계층 만들기 (Affine)

In [ ]:
class Affine:

    def __init__ (self, W, b):
        self.W = W
        self.b = b

        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None

    def forward (self, x:np.array):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        y = np.matmul(self.x, self.W)